In [2]:
from huggingface_hub import notebook_login
notebook_login()

squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16


In [3]:
import transformers
print(transformers.__version__)

from transformers.utils import send_example_telemetry
send_example_telemetry("question_answering_notebook", framework="pytorch")

# datasets transformers huggingface_hub ipywidgets

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


4.30.2


In [60]:
from datasets import load_dataset, Value

from datasets import load_dataset
datasets = load_dataset("csv", data_files="pokemon_contexts2.csv")




Found cached dataset csv (C:/Users/mvinn/.cache/huggingface/datasets/csv/default-b589ac95fffbeec3/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
import json

# new_answers = datasets['train']['answers'].copy()
# new_answers = [json.loads(answer.replace("'", '"')) for answer in new_answers]
datasets['train'] = datasets['train'].map(lambda example: print(example))
# datasets['train'] = datasets['train'].map(lambda example: { 'answers': [json.loads(answer.replace("'", '"')) for answer in example['answers']] })


# datasets['train'].cast_column('answers', json.loads)

# # for i, answer in enumerate(datasets['train']['answers']):
# #     datasets['train']['answers'][i] = json.loads(answer.replace("'", '"'))


Map:   0%|          | 0/152 [00:00<?, ? examples/s]

{'Unnamed: 0': 2, 'title': 'Bulbasaur', 'context': 'Bulbasaur its a Grass first generation Pokemon that It uses the nutrients that are stored in the seed on its back in order to grow. The reception of Bulbasaur has been largely positive and it often appears in "top Pokémon lists".[3][4] Its English name is a portmanteau of "bulb" and "dinosaur".[5]', 'question': 'Whats the type of Bulbasaur?', 'answers': "{'answer_start': 16, 'text': 'Grass'}", 'Unnamed: 5': None, 'Unnamed: 6': 298.0}
{'Unnamed: 0': 3, 'title': 'Ivysaur', 'context': 'Ivysaur its a Grass first generation Pokemon that When it\'s exposed to sunlight, the seed, now a bud, grows so much that it apparently loses the ability to stand on its hind legs. It grows by drawing in energy and when a wafting scent fills the area and it spends more time in the sun, it\'s ready to evolve. To support this weight, its legs grow thicker in size and strength. Ivysaur is a playable character in Super Smash Bros. Brawl and Super Smash Bros. U

In [54]:
datasets['train']['answers'][:5]

[[{'answer_start': 16, 'text': 'Grass'},
  {'answer_start': 14, 'text': 'Grass'},
  {'answer_start': 15, 'text': 'Grass'},
  {'answer_start': 17, 'text': 'Fire'},
  {'answer_start': 17, 'text': 'Fire'},
  {'answer_start': 16, 'text': 'Fire'},
  {'answer_start': 15, 'text': 'Water'},
  {'answer_start': 16, 'text': 'Water'},
  {'answer_start': 16, 'text': 'Water'},
  {'answer_start': 15, 'text': 'Bug'},
  {'answer_start': 14, 'text': 'Bug'},
  {'answer_start': 17, 'text': 'Bug'},
  {'answer_start': 13, 'text': 'Bug'},
  {'answer_start': 13, 'text': 'Bug'},
  {'answer_start': 15, 'text': 'Bug'},
  {'answer_start': 13, 'text': 'Normal'},
  {'answer_start': 16, 'text': 'Normal'},
  {'answer_start': 14, 'text': 'Normal'},
  {'answer_start': 14, 'text': 'Normal'},
  {'answer_start': 15, 'text': 'Normal'},
  {'answer_start': 14, 'text': 'Normal'},
  {'answer_start': 13, 'text': 'Normal'},
  {'answer_start': 12, 'text': 'Poison'},
  {'answer_start': 12, 'text': 'Poison'},
  {'answer_start': 14,

In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [50]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [51]:
max_length = 1000 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

for i, example in enumerate(datasets["train"]):
  if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
    break
example = datasets["train"][i]

In [52]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

79

In [53]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [54]:
[len(x) for x in tokenized_example["input_ids"]]

[79]

In [55]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] whats the type of missingno.? [SEP] missingno. its a bird [ nb 9 ] first generation pokemon that an error handler species, " missing number " was created to handle attempts at accessing nonexistent pokemon species. [ 64 ] [ 65 ] it is regarded as one of the most famous and popular glitches in video game history. [ 66 ] [SEP]


In [56]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (4, 5), (6, 9), (10, 14), (15, 17), (18, 25), (25, 27), (27, 28), (28, 29), (0, 0), (0, 7), (7, 9), (9, 10), (11, 14), (15, 16), (17, 21), (21, 22), (22, 23), (23, 24), (25, 26), (26, 27), (28, 33), (34, 44), (45, 52), (53, 57), (58, 60), (61, 66), (67, 74), (75, 82), (82, 83), (84, 85), (85, 92), (93, 99), (99, 100), (101, 104), (105, 112), (113, 115), (116, 122), (123, 131), (132, 134), (135, 141), (141, 144), (145, 149), (149, 151), (151, 155), (155, 156), (157, 164), (165, 172), (172, 173), (173, 174), (174, 176), (176, 177), (177, 178), (178, 180), (180, 181), (182, 184), (185, 187), (188, 196), (197, 199), (200, 203), (204, 206), (207, 210), (211, 215), (216, 222), (223, 226), (227, 234), (235, 236), (236, 239), (239, 243), (244, 246), (247, 252), (253, 257), (258, 265), (265, 266), (266, 267), (267, 269), (269, 270), (0, 0)]


In [57]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

what What


In [58]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [59]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

TypeError: string indices must be integers, not 'str'